In [1]:
# !pip install basilica
import basilica

# DO NOT EXPOSE THIS

In [2]:
API_KEY = '7f8e7b80-be40-8936-1a61-cef3d1926786'

# ---

In [3]:
conn = basilica.Connection(API_KEY)
embedding = conn.embed_sentence('hey, super-cool tweet!', model='twitter')

In [4]:
len(embedding)

768

In [5]:
# this one is fake 
Nietzsche_0 = """Many are stubborn in pursuit of the path they have taken, few in pursuit of the goal."""

Nietzsche_1 = """Whoever fights monsters should see to it that in the process he does not become a monster. 
And if you gaze long enough into an abyss, the abyss will gaze back into you."""

Nietzsche_2 = """The best author will be the one who is ashamed to become a writer."""

Nietzsche_3 = """Woman was God's second mistake."""

Nietzsche_4 = """Many are stubborn in pursuit of the path they have chosen, few in pursuit of the goal."""

Nietzsche_quotes = [Nietzsche_0, Nietzsche_1, Nietzsche_2, Nietzsche_3, Nietzsche_4]

In [7]:
import basilica
sentences = Nietzsche_quotes
with basilica.Connection(API_KEY) as c:
    embeddings = list(c.embed_sentences(sentences))
# print(embeddings)

In [13]:
# for i in range(1,6):
#     print(f"Nietzsche_{i}")

In [8]:
from scipy import spatial
print(spatial.distance.cosine(embeddings[0], embeddings[1]))
print(spatial.distance.cosine(embeddings[0], embeddings[2]))
print(spatial.distance.cosine(embeddings[0], embeddings[4]))

0.1374542495567791
0.12171451277603651
0.002116208769955774


In [9]:
# with basilica.Connection(API_KEY) as c:
#     embedding = c.embed_image_file('./cats_dogs_demo/images/dog.1.jpg')
#     print(len(embedding))

In [10]:
from six.moves import zip
import json
import os

In [11]:
EMB_DIR = '/tmp/basilica-embeddings/'
if not os.path.exists(EMB_DIR):
    os.mkdir(EMB_DIR)

IMG_DIR = './cats_dogs_demo/images/'
# API_KEY = '7f8e7b80-be40-8936-1a61-cef3d1926786'

# uncomment to run - it is long

# with basilica.Connection(API_KEY) as c:
#     filenames = os.listdir(IMG_DIR)
#     embeddings = c.embed_image_files(IMG_DIR + f for f in filenames)
#     for filename, embedding in zip(filenames, embeddings):
#         with open(EMB_DIR + filename + '.emb', 'w') as f:
#             f.write(json.dumps(embedding))
#             print(filename)

## Training a classifier
Let's train a classifier that distinguishes cat pictures from dog pictures, using sklearn.linear_model.LogisticRegression.

In [12]:
import json
import numpy as np
import os
import random
import re
import sklearn.linear_model
import sklearn.preprocessing
import time

In [13]:
EMB_DIR = './basilica-embeddings/'
files = [f for f in os.listdir(EMB_DIR)]
random.shuffle(files)

train_size = int(len(files)*0.8)
# train_size = 2400



In [14]:
x_train = np.zeros((train_size, 2048))
x_test = np.zeros((len(files)-train_size, 2048))

y_train = np.zeros(train_size, dtype=int)
y_test = np.zeros(len(files)-train_size, dtype=int)

for i in range(train_size):
    filename = files[i]
    with open(EMB_DIR + filename, 'r') as f:
        x_train[i] = json.load(f)
        y_train[i] = (0 if re.match('.*cat.*', filename) else 1)

for i in range(len(files) - train_size):
    filename = files[train_size+i]
    with open(EMB_DIR + filename, 'r') as f:
        x_test[i] = json.load(f)
        y_test[i] = (0 if re.match('.*cat.*', filename) else 1)

Finally, let's train the classifier.

In [15]:
x_train = sklearn.preprocessing.normalize(x_train)
x_test = sklearn.preprocessing.normalize(x_test)

model = sklearn.linear_model.LogisticRegression()
model.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Easy, right? Let's see how well it did:

In [16]:
print('Train accuracy: %.3f' % model.score(x_train, y_train))
print('Test accuracy: %.3f' % model.score(x_test, y_test))

Train accuracy: 0.988
Test accuracy: 0.988


In [17]:
test_proba = model.predict_proba(x_test)
collected = zip(files[train_size:], y_test, test_proba)
probabilities = [(pred[y], f) for f, y, pred in collected]
probabilities.sort()
for prob, filename in probabilities[:3]:
    print('%s: %.2f' % (filename, prob))

cat.406.jpg.emb: 0.23
cat.944.jpg.emb: 0.25
cat.201.jpg.emb: 0.31


# Nearest neighbors


In [18]:
import basilica
import json
import numpy as np
import os
import random
import re
import sklearn.decomposition
import sklearn.neighbors
import sklearn.preprocessing
import time

Next, we load up the embeddings for all of the dogs:

In [19]:
# EMB_DIR = '/tmp/basilica-embeddings/'
files = [f for f in os.listdir(EMB_DIR) if re.match('.*dog.*', f)]
random.shuffle(files)
signatures = np.zeros((len(files), 2048))
for i, filename in enumerate(files):
    with open(EMB_DIR + filename, 'r') as f:
        signatures[i] = json.load(f)

Next, let's fit a nearest neighbors model. The number of dimensions you want for nearest neighbors search is usually much smaller than the number you want for classification, so we're going to PCA down to 200 dimensions first:

In [20]:
scaler = sklearn.preprocessing.StandardScaler(with_std=False)

pca = sklearn.decomposition.PCA(n_components=200, whiten=True)

signatures = sklearn.preprocessing.normalize(signatures)

signatures = scaler.fit_transform(signatures)
signatures = pca.fit_transform(signatures)
signatures = sklearn.preprocessing.normalize(signatures)

nbrs = sklearn.neighbors.NearestNeighbors(n_neighbors=4).fit(signatures)

In [21]:
# IMG_DIR = 'images/'
target_files = ['dog.1.jpg', 'dog.2.jpg', 'dog.3.jpg']
API_KEY = '7f8e7b80-be40-8936-1a61-cef3d1926786'
with basilica.Connection(API_KEY) as c:
    targets = np.array(list(c.embed_image_files(IMG_DIR + f for f in target_files)))

targets = sklearn.preprocessing.normalize(targets)
targets = scaler.transform(targets)
targets = pca.transform(targets)
targets = sklearn.preprocessing.normalize(targets)

_, all_indices = nbrs.kneighbors(targets)
for indices in all_indices:
    print(' '.join(files[i] for i in indices))

dog.1.jpg.emb dog.603.jpg.emb dog.262.jpg.emb dog.1291.jpg.emb
dog.2.jpg.emb dog.334.jpg.emb dog.849.jpg.emb dog.230.jpg.emb
dog.3.jpg.emb dog.1157.jpg.emb dog.607.jpg.emb dog.988.jpg.emb
